In [0]:
#https://www.tensorflow.org/tutorials/keras/text_classification_with_hub
#Text classification with TensorFlow Hub: Movie review

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
import numpy as np


TensorFlow 2.x selected.


In [0]:
!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0-rc1
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [0]:
#Download the dataset
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])
(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)
"""
#I basically tried to see hpw the data was written, but it is a mess
#Its better if I follow the data adquisition and then explore its shapes.

movie_reviews = tfds.load(name="imdb_reviews")
#movie_reviews['train']
#movie_reviews['test']

print(next(iter(movie_reviews['train'].batch(10))))

#No idea how the dataset is written but this gives you the data
train_examples_batch, train_labels_batch = next(iter(movie_reviews['train'].batch(10)))

#train_labels_batch
"""

'\n#I basically tried to see hpw the data was written, but it is a mess\n#Its better if I follow the data adquisition and then explore its shapes.\n\nmovie_reviews = tfds.load(name="imdb_reviews")\n#movie_reviews[\'train\']\n#movie_reviews[\'test\']\n\nprint(next(iter(movie_reviews[\'train\'].batch(10))))\n\n#No idea how the dataset is written but this gives you the data\ntrain_examples_batch, train_labels_batch = next(iter(movie_reviews[\'train\'].batch(10)))\n\n#train_labels_batch\n'

In [0]:
#@title Build the model


In [0]:
#Note:One way to represent the text is to convert sentences into embeddings vectors. 
#We can use a pre-trained text embedding as the first layer, 
#which will have three advantages:
"""
we don't have to worry about text preprocessing,
we can benefit from transfer learning,
the embedding has a fixed size, so it's simpler to process.
"""

In [1]:
#In this tutorial we take a pre-made text embedding model
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch
hub_layer(train_examples_batch[:1]) #Hace un valor de shape (1,20),
#el 1 es por el [:1], el 20 es porque es así, probablemente por el embedidng
#Creo que con esto puedes pasar cualquier texto a un array...
#Interesante...


NameError: ignored

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer) #The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector.
#The resulting dimensions are: (num_examples, embedding_dimension)
#En este caso la embedding dim es 20, (default del embedder?)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_6 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
print(model.output_shape)
#Note: our final output shape is 1 because we want just 1 value, 0 or 1

(None, 1)


In [0]:
#@title Loss function and optimizer


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#The parameters we chose are default for a probability single classification

In [0]:
#@title Train the model


In [0]:
#Train the model for 20 epochs in mini-batches of 512 samples.
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)


Epoch 1/20
30/30 [==============================] - 7s 228ms/step - loss: 1.0445 - accuracy: 0.4669 - val_loss: 0.8593 - val_accuracy: 0.4553
Epoch 2/20
30/30 [==============================] - 6s 191ms/step - loss: 0.8033 - accuracy: 0.4857 - val_loss: 0.7554 - val_accuracy: 0.5300
Epoch 3/20
30/30 [==============================] - 6s 205ms/step - loss: 0.7210 - accuracy: 0.5592 - val_loss: 0.6916 - val_accuracy: 0.5884
Epoch 4/20
30/30 [==============================] - 6s 215ms/step - loss: 0.6593 - accuracy: 0.6211 - val_loss: 0.6385 - val_accuracy: 0.6424
Epoch 5/20
30/30 [==============================] - 6s 200ms/step - loss: 0.6065 - accuracy: 0.6733 - val_loss: 0.5931 - val_accuracy: 0.6896
Epoch 6/20
30/30 [==============================] - 7s 220ms/step - loss: 0.5614 - accuracy: 0.7180 - val_loss: 0.5528 - val_accuracy: 0.7245
Epoch 7/20
30/30 [==============================] - 6s 211ms/step - loss: 0.5178 - accuracy: 0.7595 - val_loss: 0.5105 - val_accuracy: 0.7608
Epoch 

In [0]:
#@title Evaluate the model


In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

#The previous one is literally just printing this pretty
print(results)

loss: 0.318
accuracy: 0.862
[0.3179040052452866, 0.86168]


[0.3179040052452866, 0.86168]